##Домашняя работа

###Задание: дана таблица с колонками (id, name, salary, managerId), студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.

In [ ]:
!pip install pyspark >> None

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('filter_example').getOrCreate()

data = [(1, 'Joe', 70, '3'), (2, 'Henry', 80, '4'), (3, 'Sam', 60, 'Null'), (4, 'Max', 90, 'Null')]
df = spark.createDataFrame(data, ['id', 'Name', 'Salary', 'ManagerId'])

df.show()

+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     Null|
|  4|  Max|    90|     Null|
+---+-----+------+---------+



In [ ]:
employees = df.select(
    df['Name'].alias('employee_name'),
    df['Salary'].alias('employee_salary')
)[df.ManagerId == 'Null']
employees.show()

+-------------+---------------+
|employee_name|employee_salary|
+-------------+---------------+
|          Sam|             60|
|          Max|             90|
+-------------+---------------+



In [ ]:
managers = df.select(
    df['Name'].alias('manager_name'),
    df['Salary'].alias('manager_salary')
)[df.ManagerId != 'Null']
managers.show()

+------------+--------------+
|manager_name|manager_salary|
+------------+--------------+
|         Joe|            70|
|       Henry|            80|
+------------+--------------+



In [ ]:
df_with_managers = employees.join(
    managers,
    how='left'
)
df_with_managers.show()

+-------------+---------------+------------+--------------+
|employee_name|employee_salary|manager_name|manager_salary|
+-------------+---------------+------------+--------------+
|          Sam|             60|         Joe|            70|
|          Sam|             60|       Henry|            80|
|          Max|             90|         Joe|            70|
|          Max|             90|       Henry|            80|
+-------------+---------------+------------+--------------+



In [ ]:
result = df_with_managers.filter(df_with_managers['employee_salary'] > df_with_managers['manager_salary'])

result.select(df_with_managers['employee_name']).distinct().show()

+-------------+
|employee_name|
+-------------+
|          Max|
+-------------+

